### Deliverable 2: Compile, Train and Evaluate the Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../data/charity_data_preprocessed.csv")
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [2]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(['IS_SUCCESSFUL'], 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

/var/folders/x8/8cr6lh8n1w11mfdlwx2rn3_00000gn/T/ipykernel_55023/2253336778.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL'], 1)


In [3]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

len(X_train_scaled[0])

50

In [4]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 33
hidden_nodes_layer2 = 17

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = "sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = "sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 33)                1683      
                                                                 
 dense_1 (Dense)             (None, 17)                578       
                                                                 
 dense_2 (Dense)             (None, 1)                 18        
                                                                 
Total params: 2,279
Trainable params: 2,279
Non-trainable params: 0
_________________________________________________________________


2022-04-20 18:36:48.497639: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("../checkpoints/v3/", exist_ok = True)
checkpoint_path = "../checkpoints/v3/weights.{epoch:02d}.hdf5"
cp_callback = ModelCheckpoint(filepath = checkpoint_path, verbose = 1, save_weights_only = True, save_freq = 'epoch', period = 5)

In [6]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100, callbacks = [cp_callback])

# Export our model to HDF5 file
nn.save("../models/AlphabetSoupCharity_v3.h5")

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6106 - accuracy: 0.6995
Epoch 2/100
804/804 [==============================] - 0s 602us/step - loss: 0.5752 - accuracy: 0.7217
Epoch 3/100
804/804 [==============================] - 1s 635us/step - loss: 0.5714 - accuracy: 0.7241
Epoch 4/100
804/804 [==============================] - 0s 608us/step - loss: 0.5676 - accuracy: 0.7251
Epoch 5/100
722/804 [=========================>....] - ETA: 0s - loss: 0.5643 - accuracy: 0.7253
Epoch 00005: saving model to ../checkpoints/v3/weights.05.hdf5
804/804 [==============================] - 1s 634us/step - loss: 0.5638 - accuracy: 0.7258
Epoch 6/100
804/804 [==============================] - 0s 614us/step - loss: 0.5605 - accuracy: 0.7268
Epoch 7/100
804/804 [==============================] - 1s 623us/step - loss: 0.5581 - accuracy: 0.7276
Epoch 8/100
804/804 [==============================] - 0s 612us/step - loss: 0.5561 - accuracy: 0.7291
Epoch 9/100
804/804 [=========

In [7]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5439 - accuracy: 0.7341 - 170ms/epoch - 633us/step
Loss: 0.543863832950592, Accuracy: 0.7341107726097107
